In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas
from pandas import DataFrame
import datetime

In [3]:
def news_crawler(url):
    
    res = requests.get(url)
    
    if res.status_code == 200:
        
        soup = bs(res.text, 'html.parser')
        
        news = soup.select('#main_pack > div > ul > li')

        data = []
        for each in news:
            row = []

            media = each.select('._sp_each_source')

            # 각 기사의 a 태그들에서 헤드라인과 주소를 뽑기 위함
            a_tag = each.findAll('a',{'class' : '_sp_each_title'})
            headline = a_tag[0]['title']
            news_url = a_tag[0]['href']
            summary = each.select('dd')[1]

            row.append(media[0].text.strip() if len(media[0])>0 else "")
            row.append(headline.strip() if len(headline)>0 else "")
            row.append(summary.text.strip() if len(summary)>0 else "")
            row.append(news_url.strip() if len(news_url)>0 else "")

            data.append(row)
    return data

In [4]:
def loadto_end_pages(url,keyword,start_date,end_date):
        
    chk = True
    count = 0
    while chk:
        res = requests.get(url.format(keyword,count,start_date,end_date))
        if res.status_code == 200:
            soup = bs(res.text, 'html.parser')
            
            #결과값 없는 페이지 찾기
            chk_page = soup.select('#notfound')
            if len(chk_page) == 0:
                count += 1
            else:
                chk = False
                break
     
    news_total = []    
    for i in range(0, count+1):
        news_total += news_crawler(url.format(keyword,i,start_date,end_date))
    
    return news_total
        

In [5]:
def save_to_csv(url):
    
    Q_keyword = input('검색할 키워드를 입력해주세요.(해당 키워드 관련 당일 뉴스가 없으면 오류가 납니다.): ')
    keyword = Q_keyword.replace(" ","+")
    start_date = input('검색을 원하는 시작날짜를 입력해주세요(yyyymmdd)')
    end_date = input('검색을 원하는 끝날짜를 입력해주세요(yyyymmdd)')

    
    today_news = loadto_end_pages(url,keyword,start_date,end_date)
    
    df_news_total = DataFrame(today_news)
    df_news_total.columns = ['매체명','헤드라인','내용요약','뉴스url']
    df_news_total.to_csv('./'+start_date+'-'+end_date+'_News_'+keyword+'.csv')
    
    return df_news_total    

In [ ]:
#검색어, 날짜 설정(하루단위로), page counting 필요
# sort=2 : 오래된순, sort=1 : 최신순, sort=0 : 관련도 높은순
# 지금은 sort=2 로 고정.

news_url_form = "https://search.naver.com/search.naver?ie=utf8\
&where=news&query={0}&sort=2\
&start={1}1&nso=so:da,p:from{2}to{3},a:all"

save_to_csv(news_url_form)

검색할 키워드를 입력해주세요.(해당 키워드 관련 당일 뉴스가 없으면 오류가 납니다.): 건강식품
검색을 원하는 시작날짜를 입력해주세요(yyyymmdd)20170826
검색을 원하는 끝날짜를 입력해주세요(yyyymmdd)20170829
